#Alunos
- Jerônimo Ribeiro Costa Neto - 2120524
- Lucas Martins Caetano - 2120474
- Gilso Reinaldo - 2120645
- Weller Schwartz - 2120458


In [12]:
import ipywidgets as widgets # Biblioteca para criação dos botões
from IPython.display import display, clear_output
from PIL import Image
from google.colab import files
import io
import hashlib
from stegano import lsb
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.backends import default_backend


# Gerar chaves RSA para criptografia e descriptografia
chave_privada = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)
chave_publica = chave_privada.public_key()

# Definir os botões principais para cada ação com personalização
botao_embutir = widgets.Button(
    description="Inserir texto na Imagem",
    layout=widgets.Layout(width="600px"),
    button_style='primary',  # Estilo de botão (pode ser 'primary', 'success', 'info', 'warning', 'danger')
    tooltip="Clique para embutir um texto na imagem usando esteganografia.",
    icon='file-image'  # Ícone para o botão (pode usar ícones da biblioteca FontAwesome)
)

botao_recuperar = widgets.Button(
    description="Recuperar texto da Imagem",
    layout=widgets.Layout(width="600px"),
    button_style='info',
    tooltip="Clique para recuperar um texto de uma imagem esteganografada.",
    icon='search'
)

botao_hash = widgets.Button(
    description="Gerar hash da Imagem",
    layout=widgets.Layout(width="600px"),
    button_style='success',
    tooltip="Clique para gerar o hash SHA-256 da imagem.",
    icon='hash'
)

botao_criptografar = widgets.Button(
    description="Criptografar Mensagem",
    layout=widgets.Layout(width="600px"),
    button_style='warning',
    tooltip="Clique para criptografar uma mensagem.",
    icon='lock'
)

botao_descriptografar = widgets.Button(
    description="Descriptografar Mensagem",
    layout=widgets.Layout(width="600px"),
    button_style='danger',
    tooltip="Clique para descriptografar uma mensagem.",
    icon='unlock'
)

# Exibir os botões no menu
display(botao_embutir, botao_recuperar, botao_hash, botao_criptografar, botao_descriptografar)

# Definir uma área de saída para exibir resultados e mensagens
area_exibicao = widgets.Output()
texto_mensagem = widgets.Text(description="Mensagem:")
entrada_texto_criptografado = widgets.Text(description="Criptografado (hex):")

# Função auxiliar para lidar com o upload de imagens
def carregar_imagem():
    """Solicita ao usuário o upload de uma imagem e a carrega."""
    area_exibicao.clear_output()
    with area_exibicao:
        print("\nPor favor, faça upload de uma imagem.")
    carregado = files.upload()
    for nome_arquivo in carregado.keys():
        imagem = Image.open(io.BytesIO(carregado[nome_arquivo]))
        with area_exibicao:
            print(f"\nImagem '{nome_arquivo}' carregada com sucesso.")
        return imagem

# Funções de ação para cada botão (não há alteração no funcionamento das funções)
def acao_embutir_texto(evento_botao):
    """Embutir um texto na imagem carregada usando esteganografia."""
    clear_output(wait=True)
    display(botao_embutir, botao_recuperar, botao_hash, botao_criptografar, botao_descriptografar, botao_sair, area_exibicao)
    area_exibicao.clear_output()

    with area_exibicao:
        imagem_original = carregar_imagem()
        display(texto_mensagem)

    def ao_enviar_mensagem(evento_texto):
        """Função que embute a mensagem na imagem ao ser enviada."""
        mensagem = texto_mensagem.value
        imagem_esteganografada = lsb.hide(imagem_original, mensagem)
        nome_arquivo_estego = "imagem_estego.png"
        imagem_esteganografada.save(nome_arquivo_estego)
        area_exibicao.clear_output()
        with area_exibicao:
            print("\nTexto embutido na imagem. Baixe a imagem com esteganografia:")
        files.download(nome_arquivo_estego)

    texto_mensagem.on_submit(ao_enviar_mensagem)

def acao_recuperar_texto(evento_botao):
    """Recupera um texto embutido de uma imagem carregada."""
    clear_output(wait=True)
    display(botao_embutir, botao_recuperar, botao_hash, botao_criptografar, botao_descriptografar, botao_sair, area_exibicao)
    area_exibicao.clear_output()
    with area_exibicao:
        imagem_esteganografada = carregar_imagem()
        mensagem = lsb.reveal(imagem_esteganografada)
        print(f"\nMensagem recuperada: {mensagem}")

def acao_hash(evento_botao):
    """Gera o hash SHA-256 de uma imagem carregada."""
    clear_output(wait=True)
    display(botao_embutir, botao_recuperar, botao_hash, botao_criptografar, botao_descriptografar, botao_sair, area_exibicao)
    area_exibicao.clear_output()
    with area_exibicao:
        print("\nPor favor, faça upload de uma imagem para gerar seu hash.")
    carregado = files.upload()
    for nome_arquivo in carregado.keys():
        dados_imagem = carregado[nome_arquivo]
        hash_imagem = hashlib.sha256(dados_imagem).hexdigest()
        with area_exibicao:
            print(f"\nHash SHA-256 de '{nome_arquivo}':\n {hash_imagem}")

def acao_criptografar(evento_botao):
    """Criptografa uma mensagem usando a chave pública RSA."""
    clear_output(wait=True)
    display(botao_embutir, botao_recuperar, botao_hash, botao_criptografar, botao_descriptografar, botao_sair, area_exibicao)
    area_exibicao.clear_output()
    display(texto_mensagem)

    def ao_enviar_mensagem(evento_texto):
        """Função que criptografa a mensagem ao ser enviada."""
        mensagem = texto_mensagem.value
        mensagem_criptografada = chave_publica.encrypt(
            mensagem.encode(),
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        area_exibicao.clear_output()
        with area_exibicao:
            print("\nMensagem criptografada (em hexadecimal):")
            print(mensagem_criptografada.hex())

    texto_mensagem.on_submit(ao_enviar_mensagem)

def acao_descriptografar(evento_botao):
    """Descriptografa uma mensagem criptografada em hexadecimal."""
    clear_output(wait=True)
    display(botao_embutir, botao_recuperar, botao_hash, botao_criptografar, botao_descriptografar, botao_sair, area_exibicao)
    area_exibicao.clear_output()
    display(entrada_texto_criptografado)

    def ao_enviar_criptografado(evento_texto):
        """Função que descriptografa a mensagem ao ser enviada em formato hexadecimal."""
        try:
            mensagem_criptografada = bytes.fromhex(entrada_texto_criptografado.value)
            mensagem_descriptografada = chave_privada.decrypt(
                mensagem_criptografada,
                padding.OAEP(
                    mgf=padding.MGF1(algorithm=hashes.SHA256()),
                    algorithm=hashes.SHA256(),
                    label=None
                )
            )
            area_exibicao.clear_output()
            with area_exibicao:
                print(f"\nMensagem descriptografada: {mensagem_descriptografada.decode()}")
        except ValueError:
            area_exibicao.clear_output()
            with area_exibicao:
                print("\nEntrada inválida para descriptografia. Certifique-se de que a mensagem está em formato hexadecimal.")

    entrada_texto_criptografado.on_submit(ao_enviar_criptografado)

def acao_sair(evento_botao):
    """Limpa a tela e encerra a aplicação."""
    clear_output(wait=True)
    area_exibicao.clear_output()
    with area_exibicao:
        print("\nSaindo da aplicação.")

# Associar funções aos botões
botao_embutir.on_click(acao_embutir_texto)
botao_recuperar.on_click(acao_recuperar_texto)
botao_hash.on_click(acao_hash)
botao_criptografar.on_click(acao_criptografar)
botao_descriptografar.on_click(acao_descriptografar)

# Exibir a área de saída para feedback
display(area_exibicao)


Button(button_style='primary', description='Inserir texto na Imagem', icon='file-image', layout=Layout(width='…

Button(button_style='info', description='Recuperar texto da Imagem', icon='search', layout=Layout(width='600px…

Button(button_style='success', description='Gerar hash da Imagem', icon='hash', layout=Layout(width='600px'), …

Button(button_style='warning', description='Criptografar Mensagem', icon='lock', layout=Layout(width='600px'),…

Button(button_style='danger', description='Descriptografar Mensagem', icon='unlock', layout=Layout(width='600p…

Output()